<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [13]:
from __future__ import print_function

import sys
import requests
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random

In [2]:
# url = "https://www.gutenberg.org/files/100/100-0.txt"
# r = requests.get(url)
# r.encoding = r.apparent_encoding
# data = r.text

In [3]:
if 0 == 1:
    txt_data.splitlines()
    txt_data.rstrip('\n') # didnt work, but maybe it's better with them
    txt_data.split('\n\n\n')

In [4]:
# txt_data = open('sonnets.txt', 'r').read() # test external files

In [5]:
# chars = list(set(txt_data))
# num_chars = len(chars)
# txt_data_size = len(txt_data)
# print(f'Total characters: {txt_data_size}. Unique characters: {num_chars}.')

In [6]:
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# int_to_char = dict((i, c) for i, c in enumerate(chars))
# integer_encoded = [char_to_int[i] for i in txt_data] # could be used for manual one-hot encode
# print(int_to_char)

I manually copied the block of 154 sonnets into notepad, then used find and replace to make the fancy apostrophes a normal character.

In [2]:
text = open('sonnets.txt', 'r').read()

In [ ]:
len(text.split('\n\n\n')[0:10])

In [3]:
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

len(chars)

71

In [4]:
encoded = [char_int[c] for c in text]
len(encoded)

98328

In [5]:
maxlen = 64
step = 4

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 98264


In [7]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

x.shape, y.shape

((98264, 64, 71), (98264, 71))

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0127 22:43:56.395386 11536 deprecation.py:506] From C:\Users\John\Anaconda3\envs\U4-S2-NN\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(128):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=1.0)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [15]:
model.fit(x, y,
          batch_size=128,
          epochs=32,
          callbacks=[print_callback])

Epoch 1/32
98176/98264 [============================>.] - ETA: 0s - loss: 1.8604
----- Generating text after Epoch: 0
----- Generating with seed: " shall burn:
The living record of your memory.
'Gainst death, an"
 shall burn:
The living record of your memory.
'Gainst death, an whet maye the il kyoug,
Wnonnow loventhy so beare,
Of him I you
98264/98264 [==============================] - 56s 566us/sample - loss: 1.8604
Epoch 2/32
98176/98264 [============================>.] - ETA: 0s - loss: 1.8157
----- Generating text after Epoch: 1
----- Generating with seed: "Those lines that I before have writ do lie,
Even those that said"
Those lines that I before have writ do lie,
Even those that saidet in puay bond,
  There bet thy lay, the sichnfeven in and lise
98264/98264 [==============================] - 56s 569us/sample - loss: 1.8157
Epoch 3/32
98176/98264 [============================>.] - ETA: 0s - loss: 1.7799
----- Generating text after Epoch: 2
----- Generating with seed: "me with tha

In [18]:
start_index = random.randint(0, len(text) - maxlen - 1)

generated = ''

sentence = text[start_index: start_index + maxlen]
generated += sentence

print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(2048):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_int[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, temperature=1.0)
    next_char = int_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
print()

----- Generating with seed: "t not, nor he will not be free,
For thou art covetous, and he is"
t not, nor he will not be free,
For thou art covetous, and he is cuerift,
That taking no am hate of bear ills grow,
Peiting over's every notly right lovely,
Shold duephous duth that rasicty you doth mightnece,
By thou iprift but the pleak yetw action,
To hat theum love is mirenkshill O'trood:
One happy in misher's brock I not old,
That is my jays that live if store dear by
Those heart whate oly beauty ot shald this trope:
  Then raving purvough that shouls was have years,
  And that mank wo how ingild wrulds womlly?
Fur am deep a beauty's gonetest's b:
Wilt thee oncrively self alate that both.


                    44

Bot fall staiv'st of summer the taty
Groal gutst fast dead strauge Eve shame ded,
Mideau, sweet strains at elforn see their geld,
  Kill mixe, that follows you best take her thy live.
  For ous not love shade, all that wert feld your wait:
Hank every your sweet bectioved forthi

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN